# Notebook for GPT-2 scratchwork.

In [1]:
import pandas as pd

dev_csv_path = './dev_lm_sentence_input.csv'

all_sentences = pd.read_csv(dev_csv_path)['user_candidate_transcription']
print(len(all_sentences))
num_select = 2
sentences_subset = list(all_sentences.iloc[:num_select])


3193


In [7]:
import gpt2_scores
subset_scores = gpt2_scores.score_sentences(sentences_subset, model_type = '')

gpt2
0


NameError: name 'new_model_funcs' is not defined

In [10]:
import transformers

import bert_prefix_scores
from transformers import BertForMaskedLM, BertTokenizer

# 3/11: importlib help: https://stackoverflow.com/questions/1254370/reimport-a-module-in-python-while-interactive

import importlib
importlib.reload(bert_prefix_scores)


ModuleNotFoundError: No module named 'new_models'

In [40]:

# General sanity checks for trends -- the trends don't completely match up with expectation,
#   but the ones with obvious differences (okay sentence structure vs. bad structure) do.
sentence_cases = [
    'apple toast bring not unsure test coding', # Should have high surprisal
    'the person walked down the street', # Low surprisal
    'I did not want to go to the library dolphin', # Should have medium surprisal due to last word.
    'I would have preferred to eat libraries', # Should have medium surprisal due to last word.
    'I coding test passed consequently did yes', # Should have high surprisal
    'this sentence should have a low score', # Low surprisal (or medium after observing the results, because "score" is ML-specific)
    'this sentence should have a paragraph', # Even lower surprisal -- observationally this isn't the case! Which is interesting.
]

# These results aren't really intuitive.

results = bert_prefix_scores.score_sentences(sentence_cases)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
for s, score in zip(sentence_cases, results):
    print(f'For sentence {s}')
    print(f'\tThe surprisal per word was {score / len(s.split())}')

For sentence apple toast bring not unsure test coding
	The surprisal per word was 11.624098641531807
For sentence the person walked down the street
	The surprisal per word was 2.7322870890299478
For sentence I did not want to go to the library dolphin
	The surprisal per word was 3.110466194152832
For sentence I would have preferred to eat libraries
	The surprisal per word was 3.859767641339983
For sentence I coding test passed consequently did yes
	The surprisal per word was 10.099221365792411
For sentence this sentence should have a low score
	The surprisal per word was 3.726261411394392
For sentence this sentence should have a paragraph
	The surprisal per word was 4.932441711425781


# Sanity checks

In [33]:
## More informal checks of correctness in BERT
import torch

#2/20: https://huggingface.co/transformers/quickstart.html

# Please note that, for consistency with the standard "It's time to go to the" check, I use bert-base-uncased here.
# But the actual model used for the tests is the word tokenized one.

model = BertForMaskedLM.from_pretrained('bert-large-uncased-whole-word-masking')
model.eval()
    
#2/20: https://albertauyeung.github.io/2020/06/19/bert-tokenization.html
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
def test_get_positions_from_encoded():
    
    #2/20: https://albertauyeung.github.io/2020/06/19/bert-tokenization.html
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking")

    sent = "This is great"

    enc_s, seg_s = bert_prefix_scores.get_encoded_text(sent, tokenizer)
    res_tokens, res_segs, res_next_words = bert_prefix_scores.get_positions_from_encoded(enc_s, seg_s, 103)

    expected = [
        ['[CLS]', '[MASK]', 'is', 'great', '.', '[SEP]'],
        ['[CLS]', 'this', '[MASK]', 'great', '.', '[SEP]'],
        ['[CLS]', 'this', 'is', '[MASK]', '.', '[SEP]']
    ]
    
    actual = decode_token_list(res_tokens)
    assert actual == expected
    
test_get_positions_from_encoded()

In [35]:

# 3/11: importlib help: https://stackoverflow.com/questions/1254370/reimport-a-module-in-python-while-interactive
import importlib
importlib.reload(bert_prefix_scores)

import new_model_funcs

def prefix_predictions_single(this_sentence):

    score, this_probs = bert_prefix_scores.get_bert_sentence_score(this_sentence, tokenizer, model, verifying = True )

    raw_tokens = tokenizer.tokenize(this_sentence)

    words = sentence.split()
    for idx in range(0, this_probs.shape[0]):
        # Note that need to negate surprisals to treat them like probabilities, as here.
        results, _ = bert_prefix_scores.report_mask_words(this_probs[idx], raw_tokens[:idx], tokenizer)
        print(results)

In [ ]:
# Standard sanity check
#prefix_predictions_single("It's time to go to the store") 

# Below: Trying to ensure that strange behavior
#     on always choosing the ground truth as the highest next prediction is resolved.

# 3/20: This is now fixed, there is no strange behavior.

prefix_predictions_single("apple toast bring")
prefix_predictions_single("apple pie bring")
prefix_predictions_single("apple toast bring")

In [ ]:
!python gpt2_tests.py # These might have been broken by BERT development.